#  1.句子生成器

In [391]:
yufa = '''
yuju = 问候  标点  主语  谓语  宾语
问候 = 你好 | 早安 | 晚上好 | 大家好
标点 = ， | 。 | ！
主语 = 你 | 我 | 他 
谓语 = 喜欢 | 讨厌 | 热爱 | 擅长
宾语 = 洗澡 | 游泳 | 做饭 | 打球
'''

In [320]:
import random
choice = random.choice

In [321]:
def generate(grammar,target):
    if target not in grammar:
        return target
    expand = [generate(grammar,t) for t in choice(grammar[target])]
    #print(choice(grammar[target]),1)
    return ''.join([e for e in expand if e !='null'])

In [322]:
def create_grammar(grammar_str,split='=',line_split='\n'):
    grammar_d ={}
    for line in grammar_str.split(line_split):
        if not line.strip(): continue
        exp,stmt = line.split(split)
        grammar_d[exp.strip()] = [s.split()for s in stmt.split('|')]   
    #print(grammar_d)
    return grammar_d     

In [323]:
create_grammar(yufa)

{'yuju': [['问候', '标点', '主语', '谓语', '宾语']],
 '问候': [['你好'], ['早安'], ['晚上好'], ['大家好']],
 '标点': [['，'], ['。'], ['！']],
 '主语': [['你'], ['我'], ['他']],
 '谓语': [['喜欢'], ['讨厌'], ['热爱'], ['擅长']],
 '宾语': [['洗澡'], ['游泳'], ['做饭'], ['打球']]}

In [324]:
generate(create_grammar(yufa),target = 'yuju')

'大家好，你讨厌打球'

# 2.语言模型训练

In [325]:
def generate_n(grammar,target,n=3):
    s=[]
    for sentence in [generate(grammar = grammar,target = target)for i in range(n)]:
        s.append(sentence+'/n')
     
    return s

In [326]:
generate_n(create_grammar(yufa),target = 'yuju',n=4)

['早安，我热爱游泳/n', '大家好。你喜欢做饭/n', '你好，我擅长做饭/n', '你好。他讨厌游泳/n']

In [327]:
filename = r'D:\dataset\movie_comments.csv'

In [328]:
import pandas as pd

In [329]:
content = pd.read_csv(filename,encoding = 'utf-8',low_memory = False)

In [330]:
content.head(7)

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1
5,6,https://movie.douban.com/subject/26363254/,战狼2,“犯我中华者，虽远必诛”，吴京比这句话还要意淫一百倍。,1
6,7,https://movie.douban.com/subject/26363254/,战狼2,脑子是个好东西，希望编剧们都能有。,2


In [331]:
articles = content['comment'].tolist()

In [332]:
articles[6]

'脑子是个好东西，希望编剧们都能有。'

In [333]:
import re

In [334]:
def token(string):
    return re.findall('\w+',string)

In [335]:
article_clean = [''.join(token(str(a)))for a in articles]

In [336]:
article_clean[6]

'脑子是个好东西希望编剧们都能有'

In [337]:
from collections import Counter
import jieba

In [338]:
def cut(string):
    return list(jieba.cut(string))

In [339]:
with open ('article_movie','w',encoding='utf-8')as f:
    for a in article_clean:
        f.write(a+'\n')

In [340]:
cut_string(article_clean[6])

['脑子', '是', '个', '好', '东西', '希望', '编剧', '们', '都', '能', '有']

In [341]:
TOKEN = []
for i,line in enumerate(open('article_movie',encoding = 'utf-8')):
    TOKEN += cut(line)


In [342]:
from functools import reduce
from operator import add,mul
reduce(add,[1,2,3,4])

10

In [343]:
word_count = Counter(TOKEN)

In [344]:
word_count.most_common(10)

[('的', 328262),
 ('\n', 261497),
 ('了', 102420),
 ('是', 73106),
 ('我', 50338),
 ('都', 36255),
 ('很', 34712),
 ('看', 34022),
 ('电影', 33675),
 ('也', 32065)]

In [345]:
TOKEN_2_gram = [''.join(TOKEN[i:i+2])for i in range(len(TOKEN[:-2]))]

In [346]:
TOKEN_2_gram[50:60]

['胡闹\n',
 '\n吴京',
 '吴京的',
 '的炒作',
 '炒作水平',
 '水平不输',
 '不输冯小刚',
 '冯小刚但小刚',
 '但小刚至少',
 '至少不会']

In [347]:
word_2_count = Counter(TOKEN_2_gram)

In [348]:
def pro_2(word1,word2):
    if word1+word2 in word_2_count: return word_2_count[word1+word2]/len(TOKEN_2_gram)
    else:
        return 1/len(TOKEN_2_gram)

In [349]:
pro_2('你','我')

5.0296644982288845e-05

In [350]:
pro_2('你','碗')

2.104462133150161e-07

In [351]:
def get_probability(sentence):
    words = cut(sentence)
    sentence_pro =1
    for i,word in enumerate(words[:-1]):
        next_ = words[i+1]
        probability = pro_2(word,next_)
        sentence_pro *= probability
    return sentence_pro
        

In [355]:
for sen in [generate(create_grammar(yufa,split='='),target = 'yuju')for i in range(10)]:
    print(sen,get_probability(sen))

你好。你喜欢游泳 2.8636327349946366e-25
你好！他讨厌做饭 1.9613922842429015e-27
晚上好。你喜欢做饭 3.013203327022721e-32
你好！我喜欢游泳 4.4602060543683586e-24
大家好。我擅长打球 2.476605474265251e-33
大家好！他喜欢做饭 3.4672476639713514e-32
你好。我热爱做饭 5.884176852728705e-27
大家好，你热爱游泳 4.953210948530502e-33
你好。他热爱游泳 3.922784568485803e-27
早安！你热爱做饭 1.9613922842429015e-27


# 3.获得优质的语言

In [387]:
def generate_best(gram,language_model= get_probability,num=100 ):
    sentence = []
    sentence = generate_n(create_grammar(yufa,split='='),target = 'yuju',n = num)
    pro,p1 =[],[]
    for line in sentence:
        p1 = language_model(line)
        pro.append((line,p1))
    pro = sorted(pro,key = lambda x:x[1],reverse =True)
    return pro[0]
    

In [389]:
generate_best(gram = yufa,language_model = get_probability,num =6)

('早安！他擅长洗澡/n', 1.0423844878687229e-39)